In [ ]:
#Obtener el Top 200 semanal para un país en particular 
#Test 1: Japon 
#Instalar libreria de Spotify

In [ ]:
pip install spotipy --upgrade

Requirement already up-to-date: spotipy in /usr/local/lib/python3.7/dist-packages (2.17.1)


In [ ]:
#Credenciales Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="cc9a7bb857eb4282b846a1fe077003c4",
                                                           client_secret="817188bf767444ed8f6ca6f4080db1af"))

In [ ]:
#Funciones para extraer informacion de Spotify 
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None


def get_artist_album_ids(name):
	artist = get_artist(name)
	results = sp.artist_albums(artist['id'], album_type='album', country='US')
	albums = results['items']
	album_ids = []
	for album in albums:
		album_ids.append(album['id'])

	return album_ids
def get_artist_features(name, feature='valence'):
	# features: valence (default), acousticness, danceability,
	#           energy, instrumentalness, speechiness, & tempo
	album_ids = get_artist_album_ids(name)
	features = []
	for album_id in album_ids:
		results = sp.album_tracks(album_id)
		tracks = results['items']

		for track in tracks:
			audioFeatures = sp.audio_features(track['id'])
			features.append(audioFeatures[0][feature])

	return features


In [ ]:
#Funciones para obtener el Top200
import io
import pandas as pd
import requests
import time
import tqdm


def week_dates(date, weekday=0):
    week_start = date - pd.DateOffset(weekday=weekday, weeks=1)
    week_end = date + pd.DateOffset(weekday=weekday, weeks=0)
    return week_start, week_end


def get_chart(date, region='en', freq='weekly', chart='top200'):
    chart = 'regional' if chart == 'top200' else 'viral'
    date = pd.to_datetime(date)
    if date.year < 2017:
        raise ValueError('No chart data available from before 2017')
    if freq == 'weekly':
        start, end = week_dates(date, weekday=4)
        date = f'{start.date()}--{end.date()}'
    else:
        date = f'{date.date()}'
    url = f'https://spotifycharts.com/{chart}/{region}/{freq}/{date}/download'
    data = io.StringIO(requests.get(url).text)
    try:
        df = pd.read_csv(data, skiprows=1) # Fix Spotify's Note
    except pd.errors.ParserError:
        df = 'No hay datos'
        #print('No hay datos')
    return df
    #return url
    #return data


def get_charts(start, end, region='en', freq='daily', chart='top200', sleep=1):
    sample = 'D' if freq == 'daily' else 'W'
    dfs = []
    for date in tqdm.tqdm(pd.date_range(start=start, end=end, freq=sample)):
        df = get_chart(date, region=region, freq=freq, chart=chart)
        if df is not None:
            df['date'] = date
            dfs.append(df)
            time.sleep(sleep)
    return pd.concat(dfs)

In [ ]:
#Positividad promedio para un pais entre semanas
import pandas as pd
import numpy as np
start_date = '2018-01-01'
end_date = '2018-02-28'
daterange = pd.date_range(start_date, end_date, freq='W-THU')
#Artistas = []
#Canciones = []
positividad = []
semanas = 0
total_canciones = 0 
for single_date in daterange:
   date = single_date.strftime("%Y-%m-%d")
   df1 = get_chart(date, region='co')
   if type(df1) is str:
     print('No hay datos para la semana del ', date ) 
   else:
     df1 = df1.dropna() # Quita los registros vacios
     Artistas = df1['Artist'].values
     Canciones = df1['Track Name'].values
     semanas  += 1
     for i in range(len(Artistas)):
       total_canciones += 1
       query = Canciones[i] + '&' + Artistas[i]
       results = sp.search(q=query , type= 'track', limit=3, offset=0)
       #No todas las canciones tienen positividad identificada, por ello toca 
       #Poner una restriccion sobre la busqueda
       b =  results['tracks']['items']
       if (len(b) == 0):
         continue
       else:
         a = results['tracks']['items'][0]['id']
         positividad.append(sp.audio_features(a)[0]['valence'])
positividad= np.array(positividad) 
df = pd.DataFrame()
df['pais'] = ['Colombia']
df['Pos_19'] = [positividad.mean()]
df

,pais,Pos_19
0,Colombia,0.577688


In [ ]:
df = pd.DataFrame()
df['pais'] = ['Colombia']
df['Pos_18'] = [positividad.mean()]
df.to_csv('PositividadC.csv', index=False)
df

,pais,Pos_18
0,Colombia,0.577688


In [ ]:
df1.columns
df1['Streams'].values

array([547761, 532825, 397659, 349647, 343602, 321773, 309789, 296575,
       293517, 290805, 290761, 290290, 267834, 245069, 236288, 226554,
       219749, 215378, 207583, 206838, 200749, 196815, 191334, 186118,
       180408, 177519, 171877, 163394, 163132, 161788, 161086, 160921,
       160355, 159551, 156714, 155627, 149008, 148314, 145129, 144524,
       144428, 144307, 141062, 140447, 138048, 132999, 132154, 128977,
       125007, 120812, 110283, 105000, 102643, 102002, 101309, 100059,
        98278,  95073,  93957,  88411,  88157,  88120,  85704,  84834,
        84335,  83722,  79666,  79460,  78803,  76519,  73819,  70623,
        69640,  69326,  68545,  67065,  66790,  66687,  66668,  65817,
        64298,  63596,  63046,  62976,  62961,  62871,  62468,  61433,
        60114,  60089,  59099,  58218,  58052,  57632,  56915,  55492,
        55291,  54989,  54921,  54845,  54664,  54151,  54023,  53811,
        53589,  52754,  51855,  50062,  49943,  49896,  49862,  49630,
      

In [ ]:
print(semanas, i)
#print(query)
df1 = get_chart('2017-11-09', region='jp')
df1 = df1.dropna()
Artistas = df1['Artist'].values
Canciones = df1['Track Name'].values
print(Canciones[187], Artistas[187])
query = Canciones[187] + '&' + Artistas[187]
#print(query)  
#df1
#results = sp.search(q=query , type= 'track', limit=3, offset=0)


11 187
Dirty Sexy Money (feat. Charli XCX & French Montana) David Guetta


In [ ]:
len(positividad)

713

In [ ]:
df1 = get_chart('2017-01-31', region='jp')
df1
#if type(df1) is str:
#  print('Esto puede funcionar')

'No hay datos'

In [ ]:
import numpy as np
positividad= np.array(positividad)
positividad.mean()

0.6046572230014026

In [ ]:
print('Positividad : ', positividad.mean(), 'Semanas :' , semanas, 'Total canciones :', total_canciones)

Positividad :  0.6046572230014026 Semanas : 4 Total canciones : 800


In [ ]:
#Positividad ponderada para un pais entre semanas
import pandas as pd
import numpy as np
start_date = '2018-01-01'
end_date = '2018-02-28'
daterange = pd.date_range(start_date, end_date, freq='W-THU')
#Artistas = []
#Canciones = []
positividad = []
streams =[]
semanas = 0
total_canciones = 0 
for single_date in daterange:
   date = single_date.strftime("%Y-%m-%d")
   df1 = get_chart(date, region='co')
   if type(df1) is str:
     print('No hay datos para la semana del ', date ) 
   else:
     df1 = df1.dropna() # Quita los registros vacios
     Artistas = df1['Artist'].values
     Canciones = df1['Track Name'].values
     Reproducciones = df1['Streams'].values
     semanas  += 1
     for i in range(len(Artistas)):
       total_canciones += 1
       query = Canciones[i] + '&' + Artistas[i]
       results = sp.search(q=query , type= 'track', limit=3, offset=0)
       #No todas las canciones tienen positividad identificada, por ello toca 
       #Poner una restriccion sobre la busqueda
       b =  results['tracks']['items']
       if (len(b) == 0):
         continue
       else:
         a = results['tracks']['items'][0]['id']
         positividad.append(sp.audio_features(a)[0]['valence'])
         streams.append(Reproducciones[i])
positividad= np.array(positividad)
streams = np.array(streams) 
pos_pon = np.sum(np.multiply(positividad,streams))/np.sum(streams)
pos_pon
#df = pd.DataFrame()
#df['pais'] = ['Colombia']
#df['Pos_19'] = [positividad.mean()]
#df

0.6035535437865012